In [8]:
import cv2
import numpy as np


# Car[1,0]
# Human [0,1]

train_X = []
train_Y = []

test_X = []
test_Y = []

for i in range(1,5001):
    image = cv2.imread("Obje-Veri-Seti/car/car-"+str(i) + ".jpg")
    image = cv2.resize(image, (224,224))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    Augment1 = cv2.flip(image, 1)
    a,Augment2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    a,Augment3 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    train_X.append([image])
    train_X.append([Augment1])
    train_X.append([Augment2])
    train_X.append([Augment3])
    
for i in range(1,5001):
    image = cv2.imread("Obje-Veri-Seti/human/human-"+str(i) + ".jpg")
    image = cv2.resize(image, (224,224))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    Augment1 = cv2.flip(image, 1)
    a,Augment2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    a,Augment3 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    train_X.append([image])
    train_X.append([Augment1]) 
    train_X.append([Augment2])
    train_X.append([Augment3])
    
    
for i in range(1,20001):
    train_Y.append([1,0])#Car
for i in range(1,20001):
    train_Y.append([0,1])#Human
    



In [9]:
len(train_X)

40000

In [10]:
train_X = np.array(train_X)
train_X.reshape(-1,224,224,1)
print(train_X.shape)
train_Y = np.array(train_Y)
print(train_Y.shape)

X = np.array([i[0] for i in train_X]).reshape(-1,224,224,1)

(40000, 1, 224, 224)
(40000, 2)


In [1]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.optimizers import Adam

# Sequential model
model = Sequential()

# 1.Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,1), kernel_size=(11,11),\
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalization
model.add(BatchNormalization())

# 2. Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
#  Batch Normalization
model.add(BatchNormalization())

# 3. Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalization
model.add(BatchNormalization())

# 4. Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
#  Batch Normalization 
model.add(BatchNormalization())

# 5. Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
#  Batch Normalization 
model.add(BatchNormalization())

# Flatten
model.add(Flatten())
# 1. Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# droput for prevent overfitting.
model.add(Dropout(0.4))
#  Batch Normalization 
model.add(BatchNormalization())

# 2. Layer
model.add(Dense(4096)) 
model.add(Activation('relu'))
# droput for prevent overfitting.
model.add(Dropout(0.4))
#  Batch Normalization 
model.add(BatchNormalization())

#3. Layer
model.add(Dense(1000)) # Nodelar
model.add(Activation('relu'))
# Droput eklenmesi.
model.add(Dropout(0.4))
#  droput for prevent overfitting
model.add(BatchNormalization())

# Çıkış Katmanı
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        11712     
_________________________________________________________________
activation_1 (Activation)    (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)        

In [12]:
model.compile(optimizer="Adam",
             loss="categorical_crossentropy",
             metrics = ["accuracy"])


In [13]:
model.fit(X, train_Y,epochs= 5, batch_size = 128) 

Epoch 1/5
40000/40000 [==============================] - 139s 3ms/step - loss: 0.3559 - accuracy: 0.8497
Epoch 2/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.1108 - accuracy: 0.9552
Epoch 3/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.0592 - accuracy: 0.9785
Epoch 4/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.0370 - accuracy: 0.9870
Epoch 5/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.0286 - accuracy: 0.9905


In [14]:
model.save('GTA(25.12.5).h5')